In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torch
import matplotlib.pyplot as plt

# 1个ResNet块包括两个卷积层和一个残差连接，其中输入和输出的尺寸相同。

class ResNetBlock(nn.Module):
    expansion = 4
    
    def __init__(self, in_channels, out_channels, stride):
        super(ResNetBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = nn.ReLU()(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = nn.ReLU()(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out += self.shortcut(identity)
        out = nn.ReLU()(out)

        return out


In [ ]:
class ResNet50(nn.Module):
    def __init__(self, block, layers, num_classes=2):
        super(ResNet50, self).__init__()
        
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        
    def _make_layer(self, block, out_channels, blocks, stride):
        layers = []
        
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion
        
        for i in range(1, blocks):
            layers.append(block(self.in_channels, out_channels, 1))
        
        return nn.Sequential(*layers)
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = nn.ReLU()(out)
        out = self.maxpool(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        
        return out


In [61]:
# 定义超参数
num_epochs = 10
batch_size = 16
learning_rate = 0.001

# 加载数据集
train_dir='/home/ubuntu/DeepLearning/Data/Alexnet_DataSet/train'
test_dir='/home/ubuntu/DeepLearning/Data/Alexnet_DataSet/val'


transform = transforms.Compose([transforms.Grayscale(num_output_channels=1), transforms.ToTensor()])
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True)

# 初始化模型
model = ResNet50(ResNetBlock, [3, 4, 6, 3]).to('cuda')

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 用 train_dataset.classes 
# 属性来获取数据集中的所有类别，并将其与相应的索引映射起来
for i, class_name in enumerate(train_dataset.classes):    
    print(f"Class index {i}: {class_name}")
    # Class index 0:C
    # Class index 1:N


# 创建一个SummaryWriter对象，指定要保存Tensorboard日志的目录：
writer = SummaryWriter('logs')
# 定义训练集和测试集的logger
train_logger = SummaryWriter(log_dir='logs/train')
test_logger = SummaryWriter(log_dir='logs/test')
# # 训练模型
# for epoch in range(num_epochs):
#     running_loss = 0.0
#     # correct = 0 
#     # total = 0
#     for i, (images, labels) in enumerate(train_loader):
#         images = images.to('cuda')
#         labels = labels.to('cuda')
        
#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item()
#         if (i+1) % 10 == 0:
#             print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/10:.4f}')
#             # 将训练集的loss写入TensorBoard
#             train_logger.add_scalar('loss', running_loss, epoch)
#             running_loss = 0.0

Class index 0: C
Class index 1: N


In [64]:
# 训练模型
for epoch in range(num_epochs):
    running_loss = 0.0
    running_acc = 0.0  # 初始化 running_acc 变量
    # correct = 0 
    # total = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to('cuda')
        labels = labels.to('cuda')
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # 计算准确率
        predicted = torch.argmax(outputs, 1)
        correct = (predicted == labels).sum().item()
        total = labels.size(0)
        
        running_loss += loss.item()
        running_acc += correct / total
        
        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/10:.4f}, Acc: {running_acc/10:.4f}')
            # 将训练集的loss和acc写入TensorBoard
            train_logger.add_scalar('loss', running_loss, epoch)
            train_logger.add_scalar('acc', running_acc, epoch)
            running_loss = 0.0
            running_acc = 0.0


Epoch [1/10], Step [10/399], Loss: 0.8008, Acc: 0.6250
Epoch [1/10], Step [20/399], Loss: 0.3765, Acc: 0.8812
Epoch [1/10], Step [30/399], Loss: 0.3711, Acc: 0.8438
Epoch [1/10], Step [40/399], Loss: 0.3688, Acc: 0.8688
Epoch [1/10], Step [50/399], Loss: 0.5516, Acc: 0.7125
Epoch [1/10], Step [60/399], Loss: 0.3942, Acc: 0.8562
Epoch [1/10], Step [70/399], Loss: 0.3561, Acc: 0.8750
Epoch [1/10], Step [80/399], Loss: 0.3091, Acc: 0.8750
Epoch [1/10], Step [90/399], Loss: 0.4714, Acc: 0.8500
Epoch [1/10], Step [100/399], Loss: 0.4802, Acc: 0.8187
Epoch [1/10], Step [110/399], Loss: 0.4285, Acc: 0.8250
Epoch [1/10], Step [120/399], Loss: 0.4111, Acc: 0.8500
Epoch [1/10], Step [130/399], Loss: 0.4403, Acc: 0.8250
Epoch [1/10], Step [140/399], Loss: 0.4777, Acc: 0.7688
Epoch [1/10], Step [150/399], Loss: 0.4328, Acc: 0.8187
Epoch [1/10], Step [160/399], Loss: 0.3905, Acc: 0.8562
Epoch [1/10], Step [170/399], Loss: 0.4416, Acc: 0.8187
Epoch [1/10], Step [180/399], Loss: 0.5147, Acc: 0.7875
E

In [60]:
# 测试模型
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to('cuda')
        labels = labels.to('cuda')
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        # 计算测试集的accuracy并写入TensorBoard
        accuracy = 100 * correct / total
        test_logger.add_scalar('accuracy', accuracy, epoch)
        # print(f'Accuracy of the network on the {total} test images: {100 * correct / total}%')
        # # 输出每个epoch的信息
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: train_loss{len(train_loader):.4f}, Accuracy: {accuracy}%')

Accuracy of the network on the 16 test images: 100.0%
Accuracy of the network on the 32 test images: 100.0%
Accuracy of the network on the 48 test images: 97.91666666666667%
Accuracy of the network on the 64 test images: 93.75%
Accuracy of the network on the 80 test images: 93.75%
Accuracy of the network on the 96 test images: 94.79166666666667%
Accuracy of the network on the 112 test images: 93.75%
Accuracy of the network on the 128 test images: 94.53125%
Accuracy of the network on the 144 test images: 93.05555555555556%
Accuracy of the network on the 160 test images: 92.5%
Accuracy of the network on the 176 test images: 90.3409090909091%
Accuracy of the network on the 192 test images: 90.625%
Accuracy of the network on the 208 test images: 89.42307692307692%
Accuracy of the network on the 224 test images: 89.28571428571429%
Accuracy of the network on the 240 test images: 88.75%
Accuracy of the network on the 256 test images: 85.546875%
Accuracy of the network on the 272 test images: 

In [72]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, roc_curve, auc

# 测试模型
y_true = []
y_pred = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to('cuda')
        labels = labels.to('cuda')
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        # 保存预测结果和真实标签，用于计算指标
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total

    # 计算精确率、召回率和混淆矩阵
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    cm = confusion_matrix(y_true, y_pred)

    # 计算ROC曲线和AUC
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)

    # 将指标写入TensorBoard
    test_logger.add_scalar('accuracy', accuracy, epoch)
    test_logger.add_scalar('precision', precision, epoch)
    test_logger.add_scalar('recall', recall, epoch)
    test_logger.add_scalar('roc_auc', roc_auc, epoch)
    # 可以使用 add_image() 方法将混淆矩阵转换为图像并写入TensorBoard
    # 或者使用 add_figure() 方法将 ROC曲线绘制为图像并写入TensorBoard
    # ...

    # 输出每个epoch的信息
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: train_loss{len(train_loader):.4f}, Accuracy: {accuracy}%, Precision: {precision}, Recall: {recall}, ROC AUC: {roc_auc}')


AttributeError: module 'numpy.linalg.lapack_lite' has no attribute '_ilp64'

In [ ]:
from PIL import Image
# 加载要预测的图像并进行预处理
image_path = '/home/ubuntu/DeepLearning/U-net/Pytorch-UNet/headtest.jpg'
image = Image.open(image_path).convert('L')  # 转换为灰度图像
image = transform(image).unsqueeze(0)  # 转换为张量并添加一个维度

# 将图像传递给模型进行预测
with torch.no_grad():
    output = model(image.to('cuda'))
    _, predicted = torch.max(output.data, 1)

# 输出预测结果
print('Predicted label:', predicted.item())
for i, class_name in enumerate(train_dataset.classes):
    print(f"Class index {i}: {class_name}")